In [5]:
%pip install transformers

import os
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import spacy


Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load the model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("DistilGPT-2 loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

DistilGPT-2 loaded successfully!


In [29]:
import pandas as pd
import os

json_file_path = 'feedback_comments.json'

# Check if the file exists
if not os.path.exists(json_file_path):
    raise FileNotFoundError(f"The file at {json_file_path} does not exist.")

# Check if the file is not empty
if os.path.getsize(json_file_path) == 0:
    raise ValueError(f"The file at {json_file_path} is empty.")

# Read and print the content of the file
with open(json_file_path, 'r') as file:
    content = file.read()
    # print("File content:")
    # print(content)

try:
    df = pd.read_json(json_file_path)
except ValueError as e:
    raise ValueError(f"Error reading JSON file: {e}")

print(df.sample(20))

                                              Feedback          Category
7    The video avoided unnecessary jargon, which I ...   Content Clarity
105  The visuals and animations made the content mo...        Engagement
117  I liked how the speaker used humor to make the...        Engagement
173  The examples were well-chosen and directly tie...         Relevance
53   I loved how the speaker repeated important ide...   Content Clarity
86   I appreciated how the video compared similar i...   Content Clarity
132  The conversational tone of the video made it f...        Engagement
127  The use of real-world examples made the video ...        Engagement
247  There were constant buffering issues during th...  Technical Issues
110  The pacing of the video kept it interesting wi...        Engagement
164  The speaker related the content to commonly fa...         Relevance
287  The text on the visuals was blurry and hard to...  Technical Issues
31   The consistent use of terminology made the con

In [30]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning
df['Cleaned_Feedback'] = df['Feedback'].apply(clean_text)


In [33]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/pouyasmac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:


# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Tokenize function using SpaCy
def spacy_tokenizer(text):
    doc = nlp(text)
    return [token.text for token in doc]

# Apply tokenization to the 'Cleaned_Feedback' column
df['Tokenized_Feedback'] = df['Cleaned_Feedback'].apply(spacy_tokenizer)

# Display tokenized feedback
print(df[['Cleaned_Feedback', 'Tokenized_Feedback']].head())


                                    Cleaned_Feedback  \
0       the explanation was clear and easy to follow   
1  i understood the topic much better after watch...   
2  the concepts were wellexplained with simple la...   
3  the speaker broke down complex ideas into mana...   
4  i appreciate how the examples made the content...   

                                  Tokenized_Feedback  
0  [the, explanation, was, clear, and, easy, to, ...  
1  [i, understood, the, topic, much, better, afte...  
2  [the, concepts, were, wellexplained, with, sim...  
3  [the, speaker, broke, down, complex, ideas, in...  
4  [i, appreciate, how, the, examples, made, the,...  


In [40]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the local LLM
model_name = "distilgpt2"  # Replace with your local LLM model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [44]:
def construct_prompt(feedback):
    return (f"Classify the following feedback into one of the categories:\n"
            f"Feedback: \"{feedback}\"\n"
            f"Categories: Content Clarity, Engagement, Relevance, Pacing, Technical Issues.\n"
            f"Answer:")


In [48]:
def classify_feedback(feedback):
    # Construct the prompt
    prompt = construct_prompt(feedback)

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    # Generate output with max_new_tokens
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=20,  # Limit only the generated output to 20 tokens
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the category from the response
    category = response.split("Answer:")[-1].strip()
    return category


In [49]:
# Apply classification function to each feedback in the DataFrame
df['LLM_Predicted_Category'] = df['Feedback'].apply(classify_feedback)

# Display the first few rows to verify
print(df[['Feedback', 'LLM_Predicted_Category']].head())


                                            Feedback  \
0      The explanation was clear and easy to follow.   
1  I understood the topic much better after watch...   
2  The concepts were well-explained, with simple ...   
3  The speaker broke down complex ideas into mana...   
4  I appreciate how the examples made the content...   

                              LLM_Predicted_Category  
0                     "The answer was clear and easy  
1  "I understand the topic much better after watc...  
2                                 "The concepts were  
3  "The speaker broke down complex ideas into man...  
4  "I appreciate how the examples made the conten...  
